In [ ]:
# url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/YellowLabradorLooking_new.jpg/1200px-YellowLabradorLooking_new.jpg'
# url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Cute_dog.jpg/1600px-Cute_dog.jpg?20140729055059'
url = 'https://github.com/gpu-mode/lectures/blob/main/lecture_003/puppy.jpg'

In [ ]:
import torch, os, math, gzip, pickle
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
from pathlib import Path
import requests


from torch import tensor
import torchvision as tv
import torchvision.transforms.functional as tvf
from torchvision import io
from torch.utils.cpp_extension import load_inline

In [ ]:
# path_image = Path('puppy.jpg')
# if not path_image.exists():
#     response = requests.get(url)
#     with open(path_image, 'wb') as f:
#         f.write(response.content)
#     print('Downloaded!')

path_img = Path('puppy.jpg')
if not path_img.exists(): urlretrieve(url, path_img)

In [ ]:
import os
print(os.path.getsize('puppy.jpg'), 'bytes')

# Check the first few bytes
with open('puppy.jpg', 'rb') as f:
    print(f.read(100))

In [ ]:
img = io.read_image('puppy.jpg')
print(img.shape)
img[:2, :3, :4]

In [ ]:
def show_img(x, figsize=(4,3), **kwargs):
    plt.figure(figsize=figsize)
    plt.axis('off')
    if len(x.shape)==3: x = x.permute(1, 2, 0)
    plt.imshow(x.cpu(), **kwargs)

In [ ]:
img2 = tvf.resize(img, 150, antialias=True)
ch, h, w = img2.shape
ch, h, w, h*w

In [ ]:
show_img(img2)

In [ ]:
def rgb2gray(x):
    c, h, w = x.shape
    n = h*w
    x = x.flatten()
    res = torch.empty(n, dtype=x.dtype, device=x.device)
    for i in range(n): res[i] = 0.2989*x[i] + 0.5870*x[i+n] + 0.1140*x[i+2*n]
    return res.view(h, w)



In [ ]:
%%time
img_rgb = rgb2gray(img2)


In [ ]:
show_img(img_rgb)

In [ ]:
def blk_kernel(f, blocks, threads, *args):
    for i in range(blocks):
        for j in range(threads): f(i, j, threads, *args)

In [ ]:
def rgb2gray_bk(blockidx, threadidx, blockdim, x, out, n):
    i = blockidx*blockdim + threadidx
    if i<n: out[i] =  0.2989*x[i] + 0.5870*x[i+n] + 0.1140*x[i+2*n]

In [ ]:
def rgb2gray_pybk(x):
    c, h, w = x.shape
    n = h*w
    x = x.flatten()
    res = torch.empty(n, dtype=x.dtype, device=x.device)
    threads = 256
    blocks = int(math.ceil(h*w/threads))
    blk_kernel(rgb2gray_bk, blocks, threads, x, res, n)
    return res.view(h, w)

In [ ]:
img_g = rgb2gray_pybk(img2)
show_img(img_g, cmap='gray')

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING']='1' 

In [ ]:
%pip install -q wurlitzer ninja

In [ ]:
%pip install pybind11

In [ ]:
import pybind11
print(pybind11.get_include())

In [ ]:
%load_ext wurlitzer

In [ ]:
def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False):
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs, extra_include_paths=[pybind11.get_include()],
                       extra_cuda_cflags=["-O2"] if opt else [], verbose=verbose, name="inline_ext")

In [ ]:
cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

inline unsigned int cdiv(unsigned int a, unsigned int b) { return ( a + b - 1) / b;}
'''

In [ ]:
cuda_src = cuda_begin + r'''
__global__ void rgb_to_grayscale_kernel(unsigned char* x, unsigned char* out, int n) {
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    if (i<n) out[i] = 0.2989*x[i] + 0.5870*x[i+n] + 0.1140*x[i+2*n];
}

torch::Tensor rgb_to_grayscale(torch::Tensor input) {
    CHECK_INPUT(input);
    int h = input.size(1);
    int w = input.size(2);
    printf("h*w: %d%d\n", h, w);
    auto output = torch::empty({h,w}, input.options());
    int threads = 256;
    rgb_to_grayscale_kernel<<<cdiv(w*h, threads), threads>>>(
        input.data_ptr<unsigned char>(), output.data_ptr<unsigned char>(), w*h);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [ ]:
cpp_src = "torch::Tensor rgb_to_grayscale(torch::Tensor input);"

In [ ]:
module = load_cuda(cuda_src, cpp_src, ['rgb_to_grayscale'], verbose=True)

In [ ]:
dir(module)

In [ ]:
imgc = img.contiguous().cuda()

In [ ]:
%%time
res = module.rgb_to_grayscale(imgc).cpu()
h,w = res.shape
h,w,h*w

In [ ]:
show_img(res, cmap='gray')

In [ ]:
import gzip, pickle
from urllib.request import urlretrieve
from pathlib import Path
from torch import tensor

In [ ]:
MNIST_URL = 'https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [ ]:
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, (x_train, y_train, x_valid, y_valid))
x_train.shape, x_train.type()

In [ ]:
torch.manual_seed(1)
weights = torch.randn(784, 10)
weights

In [ ]:
from types import SimpleNamespace as ns

In [ ]:
def blk_kernel2d(f, blocks, threads, *args):
    for i0 in range(blocks.y):
        for i1 in range(blocks.x):
            for j0 in range(threads.y):
                for j1 in range(threads.x): f(ns(y=i0,x=i1), ns(y=j0,x=j1), threads, *args)

In [ ]:
def matmul_bk(blockIdx, threadIdx, blockDim, m ,n, out, h, w, k):
    r = blockIdx.y*blockDim.y + threadIdx.y
    c = blockIdx.x*blockDim.x + threadIdx.x

    if (r >= h or c >= w): return
    o = 0
    for i in range(k): o += m[r*k + i] * n[i*w + c]
    out[r*w + c] = o

In [ ]:
def matmul_2d(m, n):
    h,k = m.shape
    k2,w = n.shape
    assert k==k2, "size mismatch"
    output = torch.zeros(h, w, dtype=m.dtype)
    tpb = ns(x=16,y=16)
    blocks = ns(x=math.ceil(w/tpb.x), y=math.ceil(h/tpb.y))
    blk_kernel2d(matmul_bk, blocks, tpb, 
                 m.flatten(), n.flatten(), output.flatten(), h, w, k)

In [ ]:
m1 = torch.randn(500, 200, device="cuda")
m2 = torch.randn(200, 1000, device="cuda")
# res = matmul_2d(m1, m2)
# res

In [ ]:
cuda_src = cuda_begin + r'''
__global__ void matmul_k(float* m, float* n, float* out, int h, int k, int w) {
    int r = blockIdx.y*blockDim.y + threadIdx.y;
    int c = blockIdx.x*blockDim.x + threadIdx.x;

    if (r>=h || c>=w) return;
    float o = 0;
    for(int i=0; i < k; i++) o += m[r*k+i]*n[i*w+c];
    out[r*w+c] = o;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "Size mismatch!");
    auto output = torch::zeros({h,w}, m.options());

    dim3 tpb(16,16);
    dim3 blocks(cdiv(w,tpb.x), cdiv(h, tpb.y));
    matmul_k<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [ ]:
cpp_src = "torch::Tensor matmul(torch::Tensor m, torch::Tensor n);"

In [ ]:
module = load_cuda(cuda_src, cpp_src, ['matmul'])

In [ ]:
m1c, m2c = m1.contiguous().cuda(), m2.contiguous().cuda()

In [ ]:
%time module.matmul(m1,m2)


In [ ]:
p = torch.cuda.get_device_properties(0)

In [ ]:
p.max_threads_per_multi_processor

In [ ]:
cuda_src = cuda_begin + r'''
constexpr int TILE_SIZE = 16;

__global__ void tiled_matmul_kernel(float* out, float* M, float*N, int h, int w, int k) {
    __shared__ float Ms[TILE_SIZE][TILE_SIZE];
    __shared__ float Ns[TILE_SIZE][TILE_SIZE];

    int r = blockIdx.y*blockDim.y + threadIdx.y;
    int c = blockIdx.x*blockDim.x + threadIdx.x;

    float res = 0.0f;

    for (int K_tileidx = 0; K_tileidx < (k + TILE_SIZE - 1)/TILE_SIZE; K_tileidx++) {
        Ms[threadIdx.y][threadIdx.x] = (((r < h) && (K_tileidx*TILE_SIZE + threadIdx.x < k)) ? M[r*k + (K_tileidx*TILE_SIZE + threadIdx.x)]: 0.0f);
        Ns[threadIdx.y][threadIdx.x] = (((K_tileidx*TILE_SIZE + threadIdx.y < k) && (c < w)) ? N[(K_tileidx*TILE_SIZE + threadIdx.y)*w + c]: 0.0f);
        __syncthreads(); // all the tile is filled after this as all threads in the tile has finished work

        for (int idx=0; idx < TILE_SIZE; idx++) {
            res += Ms[threadIdx.y][idx] * Ns[idx][threadIdx.x];
        }
        __syncthreads(); // wait for all the threads in the tile to finish work on the shared mem

    }

    if ((r<h) && (c<w)) {
        out[r*w + c] = res;
    }
}

torch::Tensor titled_matmul(const torch::Tensor& m, const torch::Tensor& n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "size mismatch");
    auto output = torch::empty({h,w}, m.options());

    dim3 tpb(TILE_SIZE, TILE_SIZE);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));
    tiled_matmul_kernel<<<blocks, tpb>>>(output.data_ptr<float>(), m.data_ptr<float>(), n.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

cpp_src = """
torch::Tensor titled_matmul(const torch::Tensor& m, const torch::Tensor& n);
"""
titled_matmul_module = load_cuda(cuda_src, cpp_src, ['titled_matmul'])

In [ ]:
dir(titled_matmul_module)

In [ ]:
dir(module)

In [ ]:
aa = torch.randn(500, 200, device="cuda")
bb = torch.randn(200, 1000, device="cuda")

%timeit titled_matmul_module.titled_matmul(aa, bb)

%timeit aa@bb

(titled_matmul_module.titled_matmul(aa, bb) - aa@bb).abs().max()
